In [2]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-

In [3]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### Load numpy array

In [15]:
text_npy = np.load('./npy/text.npy')
poster_npy = np.load('./npy/poster.npy')
IGimg_npy = np.load('./npy/IGimg.npy')
movie_genre = np.load('./npy/movie_genre.npy')
# usr_following = np.load()
# usr_genre = np.load()

print('Text:', text_npy.shape)
print('Poster:', poster_npy.shape)
print('IGimg:', IGimg_npy.shape)
print('Movie genre:', movie_genre.shape)
# print('User following:', usr_following.shape)
# print('User genre:', usr_genre.shape)

Text: (166, 300)
Poster: (166, 256)
IGimg: (166, 256)
Movie genre: (166, 20)


# Training & testing split

### Setup 

In [11]:
usr_test_amount = 200 # int(usr_npy.shape[0] * 0.1)
movie_test_amount = 30 # int(movie_npy.shape[0] * 0.2)

print(usr_test_amount, movie_test_amount)

200 30


In [ ]:
def trainTestSplit(usr_arr):
    print('Oringinal shape: {}'.format(usr_arr.shape))
    

In [ ]:
train_idx, test_idx = trainTestSplit(usr_npy)

# Recommendation model

In [ ]:
usr_nb = 1489 # the number of users
movie_nb = 88  # the number of movies
laten_dim = 32 # latent dims
ft_dim = 2048 # feature dims